# GBDT算法 Gradient Boosting Decision Tree

GBDT也是集成学习Boosting的一种算法，但是却和传统的Adaboost有很大的不同。在GBDT的迭代中，假设我们前一轮迭代得到的强学习器是$f_{t-1}(x)$，损失函数是$L(y,f_{t-1}(x))$，迭代的目标是找到一个`CART`回归树模型的弱学习器$h_t(x)$，让本轮的损失函数$L(y,f_t(x)) = L(y,f_{t-1}(x)+h_t(x))$`最小`。即每次迭代找到的决策树要让损失函数值更小。

Xgboost（Extreme）在GBDT的基础上增加了正则化项来控制模型的复杂度，提高模型泛化能力，并且可以选择更多的不限于CART的基础模型。

随即森林是构造做个基础的决策树模型，`并行`训练多个决策树模型，将各个决策树模型的最终结果取均值或则最终的决策结果，而Xgboost是在决策树模型中，先构造第一棵树，再依次构造并加入其他的树，是一个`串行`算法。

> 有科学家证明决策树模型是一个很好的基础模型。

## Xgboost 算法

Xgboost损失函数最终为：

$$
    L_t=\sum_{i=1}^mL(y_i, f_{t-1}(x_i)+ h_t(x_i)) + \gamma J + \frac{\lambda}{2}\sum_{j=1}^Jw_{tj}^2
$$

其中，L是损失函数，可以是平方损失函数或者logistics损失函数等，J为决策树t的叶子节点的个数，w是叶子节点的最优值，参数$\gamma$是限制叶子节点个数，$\lambda$是正则化惩罚力度。

利用二阶泰勒展开，得到一个近似解，一阶导数和二阶导数转化合并为$g_i$和$h_i$：

$$
    L_t \approx \sum_{i=1}^m( L(y_i, f_{t-1}(x_i)) + g_{ti}h_t(x_i) + \frac{1}{2} h_{ti} h_t^2(x_i)) +  \gamma J + \frac{\lambda}{2}\sum_{j=1}^Jw_{tj}^2
$$

除去损失函数进一步简化目标函数，再将原本按样本遍历，转化为按叶子节点遍历：

$$
    L_t  =  \sum_{j=1}^J [G_{tj}w_{tj} + \frac{1}{2}(H_{tj}+\lambda)w_{tj}^2] + \gamma J
$$

其中$G_{tj}$和$H_{tj}$利用一阶和二阶导函数计算得到。

到此为止得到了最终的损失函数，那么最小化损失函数就可以得到叶子区域的最优解$w_{tj}$。

对损失函数求导且令其为0，得到最优解$w_{tj}$：

$$
    w_{tj} = - \frac{G_{tj}}{H_{tj} + \lambda}
$$

注意到在$w_{tj}$取最优解的时候，原损失函数对应的表达式为：

$$
    L_t = -\frac{1}{2} \sum_{j=1}^J\frac{G_{tj}^2}{H_{tj} + \lambda} +\gamma J
$$

目标函数中的$G/(H+\lambda)$部分，表示着每一个叶子节点对当前模型损失的贡献程度，得到Gain的计算表达式：

$$
    \max \frac{1}{2}\frac{G_L^2}{H_L + \lambda} + \frac{1}{2}\frac{G_R^2}{H_R+\lambda}  - \frac{1}{2}\frac{(G_L+G_R)^2}{H_L+H_R+ \lambda} - \gamma
$$


## Xgboost算法流程

> 输入是训练集样本$I=\{(x_1,y_1),(x_2,y_2),\dots,(x_m,y_m)\}$，最大迭代次数T, 损失函数L，正则化系数λ,γ。
> 
> 输出是强学习器f(x)

对迭代轮数$t=1,2,\dots,T$有：
1. 计算第i个样本$(i=1,2,\dots,m)$在当前轮基于$f_{t-1}(x_i)$的一阶导数$g_{ti}$和二阶导数$h_{ti}$的损失函数L，计算所有样本的一阶导数和$G_t = \sum_{i=1}^m g_{ti}$,二阶导数和$H_t = \sum_{i=1}^m h_{ti}$。
2. 基于当前节点尝试分裂决策树，默认分数score=0，G和H为当前需要分裂的节点的一阶二阶导数之和。

   对特征序号$k=1,2,\dots,K$:

   1. $G_L=0, H_L=0$
   2. 1. 将样本`按特征k里的值从小到大排列`，线性扫描样本，依次计算当前样本放入左子树后，左右子树一阶和二阶导数和：
      $$
          G_L = G_L+ g_{ti}, G_R=G-G_L \\
          H_L = H_L+ h_{ti}, H_R=H-H_L
      $$
   2. 2. 根据`贪心算法`，尝试更新最大的分数：
      $$
          score = max(score, \frac{1}{2} \frac{G_L^2}{H_L + \lambda} + \frac{1}{2}\frac{G_R^2}{H_R+\lambda}  - \frac{1}{2} \frac{(G_L+G_R)^2}{H_L+H_R+ \lambda} -\gamma )
      $$
3. 基于最大score对应的划分特征和特征值分裂子树。
4. 如果最大score为0，则当前决策树建立完毕，计算所有叶子区域的$w_{tj}$, 得到弱学习器$h_t(x)$，更新强学习器$f_t(x)$,进入下一轮弱学习器迭代.如果最大score不是0，则转到第2步继续尝试分裂决策树。

## xgboost特点

* 优势
    1. 而 Xgboost 在 GBDT 的基础上对损失函数进行了二阶泰勒展开，一方面增加精度，另一方面二阶泰勒展开可以近似大量损失函数，从而进行使用；
    2. Xgboost 不仅支持 CART 还支持其他基础分类器；
    3. 在目标函数中加入了正则项，防止过拟合，控制模型复杂度；
    4. 对于特征的值有缺失的样本，Xgboost 采用的稀疏感知算法可以自动学习出它的分裂方向，很好的自动处理缺失值；
    5. 在特征粒度上进行并行计算（Xgboost 耗时主要在对特征的值进行排序）
* 劣势
    1. 预排序过程的空间复杂度过高，不仅需要存储特征值，还需要存储特征对应样本的梯度统计值的索引，相当于消耗了两倍的内存。

> 可以看出，Xgboost 的缺点相对于优势来说很小，也是xgboost被称为kaggle神器的原因。